# import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_NU M_THREADS"] = "2"
os.environ["OMP_NUM_THREADS"] = "2"
import time

import numpy
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter

In [3]:
from ubs8k.datasetManager import DatasetManager
from ubs8k.datasets import Dataset

import sys
sys.path.append("../..")

from util.utils import reset_seed, get_datetime, get_model_from_name
from util.checkpoint import CheckPoint
from metric_utils.metrics import CategoricalAccuracy, FScore, ContinueAverage

# Arguments

In [4]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("-d", "--dataset_root", default="../../datasets/ubs8k", type=str)
parser.add_argument("--supervised_ratio", default=1.0, type=float)
parser.add_argument("-t", "--train_folds", nargs="+", default=[1, 2, 3, 4, 5, 6, 7, 8, 9], type=int)
parser.add_argument("-v", "--val_folds", nargs="+", default=[10], type=int)

parser.add_argument("--model", default="cnn0", type=str)
parser.add_argument("--batch_size", default=32, type=int)
parser.add_argument("--nb_epoch", default=100, type=int)
parser.add_argument("--learning_rate", default=0.003, type=int)

parser.add_argument("--checkpoint_path", default="../../model_save/ubs8k/full_supervised", type=str)
parser.add_argument("--resume", action="store_true", default=False)
parser.add_argument("--tensorboard_path", default="../../tensorboard/ubs8k/full_supervised", type=str)
parser.add_argument("--tensorboard_sufix", default="", type=str)

args = parser.parse_args("")

# initialisation

In [5]:
reset_seed(1234)


# Prepare the dataset

In [6]:
audio_root = os.path.join(args.dataset_root, "audio")
metadata_root = os.path.join(args.dataset_root, "metadata")

manager = DatasetManager(
    metadata_root, audio_root,
    folds=(1,2,3,4,5,6,7,8,9,10),
    verbose=2
)

In [7]:
# prepare the sampler with the specified number of supervised file
train_dataset = Dataset(manager, folds=(1, 2, 3, 4, 5, 6, 7, 8, 9), cached=True)
val_dataset = Dataset(manager, folds=(10, ), cached=True)

# Prep model

In [8]:
torch.cuda.empty_cache()

model_func = get_model_from_name(args.model)
model = model_func()


In [9]:
from torchsummaryX import summary
input_tensor = torch.zeros((1, 64, 173), dtype=torch.float)

s = summary(model, input_tensor)


                               Kernel Shape      Output Shape   Params  \
Layer                                                                    
0_features.0.Conv2d_0         [1, 24, 3, 3]  [1, 24, 64, 173]    240.0   
1_features.0.MaxPool2d_1                  -   [1, 24, 16, 86]        -   
2_features.0.BatchNorm2d_2             [24]   [1, 24, 16, 86]     48.0   
3_features.0.ReLU6_3                      -   [1, 24, 16, 86]        -   
4_features.1.Conv2d_0        [24, 48, 3, 3]   [1, 48, 16, 86]  10.416k   
5_features.1.MaxPool2d_1                  -    [1, 48, 4, 43]        -   
6_features.1.BatchNorm2d_2             [48]    [1, 48, 4, 43]     96.0   
7_features.1.ReLU6_3                      -    [1, 48, 4, 43]        -   
8_features.2.Conv2d_0        [48, 48, 3, 3]    [1, 48, 4, 43]  20.784k   
9_features.2.MaxPool2d_1                  -    [1, 48, 1, 21]        -   
10_features.2.BatchNorm2d_2            [48]    [1, 48, 1, 21]     96.0   
11_features.2.ReLU6_3                 

## Prep training

In [10]:
# create model
torch.cuda.empty_cache()

model = model_func()
model.cuda()

cnn0(
  (features): Sequential(
    (0): ConvPoolReLU(
      (0): Conv2d(1, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2d(kernel_size=(4, 2), stride=(4, 2), padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU6(inplace=True)
    )
    (1): ConvPoolReLU(
      (0): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2d(kernel_size=(4, 2), stride=(4, 2), padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU6(inplace=True)
    )
    (2): ConvPoolReLU(
      (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2d(kernel_size=(4, 2), stride=(4, 2), padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU6(inplace=True)
   

In [11]:
s_idx, u_idx = train_dataset.split_s_u(args.supervised_ratio)
S_sampler = torch.utils.data.SubsetRandomSampler(s_idx)

training_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, sampler=S_sampler)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args.batch_size, shuffle=True)

# training parameters

In [12]:
# tensorboard
title = "%s_%s_%.1f" % (get_datetime(), model_func.__name__, args.supervised_ratio)
tensorboard = SummaryWriter(log_dir="%s/%s" % (args.tensorboard_path, title), comment=model_func.__name__)

# losses
loss_ce = nn.CrossEntropyLoss(reduction="mean")

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

# callbacks
lr_lambda = lambda epoch: (1.0 + numpy.cos((epoch-1)*numpy.pi/args.nb_epoch))
lr_scheduler = LambdaLR(optimizer, lr_lambda)

# Checkpoint
checkpoint = CheckPoint(model, optimizer, mode="max", name="%s_%s.torch" % (args.checkpoint_path, title))

# Metrics
fscore_fn = FScore()
acc_fn = CategoricalAccuracy()
avg = ContinueAverage()

reset_metrics = lambda : [m.reset() for m in [fscore_fn, acc_fn, avg]]

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

## Can resume previous training

In [13]:
if args.resume:
    checkpoint.load_last()

## training function

In [14]:
UNDERLINE_SEQ = "\033[1;4m"
RESET_SEQ = "\033[0m"


header_form = "{:<8.8} {:<6.6} - {:<6.6} - {:<8.8} {:<6.6} - {:<9.9} {:<12.12}| {:<9.9}- {:<6.6}"
value_form  = "{:<8.8} {:<6} - {:<6} - {:<8.8} {:<6.4f} - {:<9.9} {:<10.4f}| {:<9.4f}- {:<6.4f}"

header = header_form.format(
    "", "Epoch", "%", "Losses:", "ce", "metrics: ", "acc", "F1 ","Time"
)


train_form = value_form
val_form = UNDERLINE_SEQ + value_form + RESET_SEQ

print(header)

         Epoch  - %      - Losses:  ce     - metrics:  acc         | F1       - Time  


In [15]:
def train(epoch):
    start_time = time.time()
    print("")

    reset_metrics()
    model.train()

    for i, (X, y) in enumerate(training_loader):
        X = X.cuda()
        y = y.cuda()

        logits = model(X)
        loss = loss_ce(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.set_grad_enabled(False):
            pred = torch.softmax(logits, dim=1)
            pred_arg = torch.argmax(logits, dim=1)
            y_one_hot = F.one_hot(y, num_classes=10)

            acc = acc_fn(pred_arg, y).mean
            fscore = fscore_fn(pred, y_one_hot).mean
            avg_ce = avg(loss.item()).mean

            # logs
            print(train_form.format(
                "Training: ",
                epoch + 1,
                int(100 * (i + 1) / len(training_loader)),
                "", avg_ce,
                "", acc, fscore,
                time.time() - start_time
            ), end="\r")

    tensorboard.add_scalar("train/Lce", avg_ce, epoch)
    tensorboard.add_scalar("train/f1", fscore, epoch)
    tensorboard.add_scalar("train/acc", acc, epoch)

In [18]:
def val(epoch):
    start_time = time.time()
    print("")
    reset_metrics()
    model.eval()

    for i, (X, y) in enumerate(val_loader):
        X = X.cuda()
        y = y.cuda()

        logits = model(X)
        loss = loss_ce(logits, y)

        with torch.set_grad_enabled(False):
            pred = torch.softmax(logits, dim=1)
            pred_arg = torch.argmax(logits, dim=1)
            y_one_hot = F.one_hot(y, num_classes=10)

            acc = acc_fn(pred_arg, y).mean
            fscore = fscore_fn(pred, y_one_hot).mean
            avg_ce = avg(loss.item()).mean

            # logs
            print(val_form.format(
                "Validation: ",
                epoch + 1,
                int(100 * (i + 1) / len(val_loader)),
                "", avg_ce,
                "", acc, fscore,
                time.time() - start_time
            ), end="\r")

    tensorboard.add_scalar("val/Lce", avg_ce, epoch)
    tensorboard.add_scalar("val/f1", fscore, epoch)
    tensorboard.add_scalar("val/acc", acc, epoch)
    
    tensorboard.add_scalar("hyperparameters/learning_rate", get_lr(optimizer), epoch)

    checkpoint.step(acc)
    lr_scheduler.step()

In [ ]:
print(header)

start_epoch = checkpoint.epoch_counter
end_epoch = args.nb_epoch

for e in range(start_epoch, args.nb_epoch):
    train(e)
    val(e)

         Epoch  - %      - Losses:  ce     - metrics:  acc         | F1       - Time  

Training 2      - 100    -          1.1508 -           0.6005    | 0.5559   - 3.1111
Validati 2      - 100    -          1.3436 -           0.5620    | 0.5206   - 0.1170
 better performance: saving ...

Training 3      - 100    -          0.9514 -           0.6865    | 0.6666   - 2.9283
Validati 3      - 100    -          1.0552 -           0.6787    | 0.6693   - 0.1118
 better performance: saving ...

Training 4      - 100    -          0.8031 -           0.7365    | 0.7289   - 3.1884
Validati 4      - 100    -          1.8476 -           0.4785    | 0.4863   - 0.1242
Training 5      - 100    -          0.7018 -           0.7730    | 0.7658   - 3.1048
Validati 5      - 100    -          1.4411 -           0.5468    | 0.5525   - 0.1102
Training 6      - 100    -          0.6408 -           0.7943    | 0.7900   - 3.0933
Validati 6      - 100    -          1.3577 -           0.6558    | 0.6712   - 0.1

# ♫♪.ılılıll|̲̅̅●̲̅̅|̲̅̅=̲̅̅|̲̅̅●̲̅̅|llılılı.♫♪